In [1]:
import tensorflow as tf
import os

In [5]:
from PIL import Image
import numpy as np
import cv2

In [10]:
def convert_to_opencv(image):
    # RGB -> BGR conversion is performed as well.
    r,g,b = np.array(image).T
    opencv_image = np.array([b,g,r]).transpose()
    return opencv_image

def crop_center(img,cropx,cropy):
    h, w = img.shape[:2]
    startx = w//2-(cropx//2)
    starty = h//2-(cropy//2)
    return img[starty:starty+cropy, startx:startx+cropx]

def resize_down_to_1600_max_dim(image):
    h, w = image.shape[:2]
    if (h < 1600 and w < 1600):
        return image

    new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
    return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)

def resize_to_256_square(image):
    h, w = image.shape[:2]
    return cv2.resize(image, (224, 224), interpolation = cv2.INTER_LINEAR)

def update_orientation(image):
    exif_orientation_tag = 0x0112
    if hasattr(image, '_getexif'):
        exif = image._getexif()
        if (exif != None and exif_orientation_tag in exif):
            orientation = exif.get(exif_orientation_tag, 1)
            # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
            orientation -= 1
            if orientation >= 4:
                image = image.transpose(Image.TRANSPOSE)
            if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)
            if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

In [7]:
imageFile = "test.jpg"
image = Image.open(imageFile)

# Update orientation based on EXIF tags, if the file has orientation info.
image = update_orientation(image)

# Convert to OpenCV format
image = convert_to_opencv(image)

In [8]:
# If the image has either w or h greater than 1600 we resize it down respecting
# aspect ratio such that the largest dimension is 1600
image = resize_down_to_1600_max_dim(image)

In [9]:
# We next get the largest center square
h, w = image.shape[:2]
min_dim = min(w,h)
max_square_image = crop_center(image, min_dim, min_dim)

In [12]:
# Resize that square down to 256x256
augmented_image = resize_to_256_square(max_square_image)

In [29]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ["serve"], "tmp_dir/1556141025")
    graph = tf.get_default_graph()
    #print(graph.get_operations())
    for op in graph.get_operations():
        print(str(op.name))
    prob_tensor = sess.graph.get_tensor_by_name('act_softmax/truediv:0')
    predictions, = sess.run(prob_tensor, {'input_1:0': [augmented_image] })

INFO:tensorflow:Restoring parameters from tmp_dir/1556141025/variables/variables
input_1
conv1_pad/Pad/paddings
conv1_pad/Pad
conv1/kernel/Initializer/random_uniform/shape
conv1/kernel/Initializer/random_uniform/min
conv1/kernel/Initializer/random_uniform/max
conv1/kernel/Initializer/random_uniform/RandomUniform
conv1/kernel/Initializer/random_uniform/sub
conv1/kernel/Initializer/random_uniform/mul
conv1/kernel/Initializer/random_uniform
conv1/kernel
conv1/kernel/IsInitialized/VarIsInitializedOp
conv1/kernel/Assign
conv1/kernel/Read/ReadVariableOp
conv1/dilation_rate
conv1/Conv2D/ReadVariableOp
conv1/Conv2D
conv1_bn/gamma/Initializer/ones
conv1_bn/gamma
conv1_bn/gamma/IsInitialized/VarIsInitializedOp
conv1_bn/gamma/Assign
conv1_bn/gamma/Read/ReadVariableOp
conv1_bn/beta/Initializer/zeros
conv1_bn/beta
conv1_bn/beta/IsInitialized/VarIsInitializedOp
conv1_bn/beta/Assign
conv1_bn/beta/Read/ReadVariableOp
conv1_bn/moving_mean/Initializer/zeros
conv1_bn/moving_mean
conv1_bn/moving_mean/IsIn

save/RestoreV2/tensor_names
save/RestoreV2/shape_and_slices
save/RestoreV2
save/NoOp
save/NoOp_1
save/NoOp_2
save/NoOp_3
save/NoOp_4
save/NoOp_5
save/NoOp_6
save/NoOp_7
save/NoOp_8
save/NoOp_9
save/NoOp_10
save/NoOp_11
save/NoOp_12
save/NoOp_13
save/NoOp_14
save/NoOp_15
save/NoOp_16
save/NoOp_17
save/NoOp_18
save/NoOp_19
save/NoOp_20
save/NoOp_21
save/NoOp_22
save/NoOp_23
save/NoOp_24
save/NoOp_25
save/NoOp_26
save/NoOp_27
save/NoOp_28
save/NoOp_29
save/NoOp_30
save/NoOp_31
save/NoOp_32
save/NoOp_33
save/NoOp_34
save/NoOp_35
save/NoOp_36
save/NoOp_37
save/NoOp_38
save/NoOp_39
save/Identity
save/AssignVariableOp
save/NoOp_40
save/Identity_1
save/AssignVariableOp_1
save/Identity_2
save/AssignVariableOp_2
save/Identity_3
save/AssignVariableOp_3
save/Identity_4
save/AssignVariableOp_4
save/NoOp_41
save/Identity_5
save/AssignVariableOp_5
save/NoOp_42
save/Identity_6
save/AssignVariableOp_6
save/Identity_7
save/AssignVariableOp_7
save/Identity_8
save/AssignVariableOp_8
save/Identity_9
save/A

AbortedError: Operation received an exception:Status: 3, message: could not create a dilated convolution forward descriptor, in file tensorflow/core/kernels/mkl_conv_ops.cc:1111
	 [[node conv1/Conv2D (defined at <ipython-input-29-41058d9c3492>:2) ]]

Caused by op 'conv1/Conv2D', defined at:
  File "/Users/sunithapriyadarshini/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sunithapriyadarshini/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sunithapriyadarshini/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sunithapriyadarshini/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/sunithapriyadarshini/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File 